# ADS API - Copernicus Atmosphere Monitoring Service (CAMS)

Reference material:<br> 
GitHub Repository containing examples from ECMWF: https://github.com/ecmwf-projects/copernicus-training

Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/2021-02-Copernicus-ECMWF-data-tutorial.ipynb

Link to Copernicus Services:
- Link to ADS Atmosphere Copernicus: https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview
- Link to CDS Climate Copernicus: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview

Change key/value provided in the .cdsapirc file as explained in the following links:
- Climate: ERA5 https://cds.climate.copernicus.eu/api-how-to
- Atmosphere: CAMS https://ads.atmosphere.copernicus.eu/api-how-to 

Or use the Key/Value in the API request:
<br>
For ADS Data:
- url: https://ads.atmosphere.copernicus.eu/api/v2


For CDS Data:<br>
- url: https://cds.climate.copernicus.eu/api/v2


### Import libraries

In [1]:
import cdsapi
import os
import geopandas as gpd
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
cwd = os.getcwd()

### Set and end date:

Modify **date.json** file to change the date:

In [ ]:
d = open('date.json')
date = json.load(d)
start_date = date["start_date"]
end_date = date["end_date"]

### Import Bounding Box:

In [4]:
area_path = cwd + '/bounding_box/bounding_box_buffer20.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds

In [5]:
print(bounds)

[ 8.22236763 44.46930839 11.7723925  46.81902056]


Link to ADS API from (https://ads.atmosphere.copernicus.eu/api-how-to). Use the API **id** and **key** to access the **cdsapi** (put in the **keys.json** file):

In [6]:
f = open('keys.json')
keys = json.load(f)

In [8]:
c_atm = cdsapi.Client(keys["ATM_ID"], keys["ATM_KEY"])

- - -

# ADS Atmosphere Copernicus

# [CAMS - PM2.5 - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for PM2.5 over the study area:

In [9]:
pm25_path = cwd+r'\temp\pm25_cams.nc'
pm25 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'particulate_matter_2.5um',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    pm25_path
    )

2022-03-03 15:24:41,963 INFO Welcome to the CDS
2022-03-03 15:24:41,965 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:24:42,015 INFO Request is queued
2022-03-03 15:24:43,071 INFO Request is running
2022-03-03 15:27:33,614 INFO Request is completed
2022-03-03 15:27:33,616 INFO Downloading https://download-0001.copernicus-atmosphere.eu/cache-compute-0001/cache/data9/adaptor.cams_regional_fc.retrieve-1646317608.0364401-8196-1-944edae7-d6d7-40b6-9ca4-bcac3fbadd9e.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\pm25_cams.nc (2.2M)
2022-03-03 15:27:34,201 INFO Download rate 3.8M/s   


Read pm25 .nc file:

In [10]:
pm25 = xr.open_dataset(pm25_path)

In [11]:
pm25_mean = pm25.mean(dim='time')

Write pm25_mean .nc file:

In [12]:
pm25_mean.to_netcdf(cwd+r'\temp\pm25_cams.nc')

- - -

# [CAMS - Ammonia - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for ammonia (NH3) over the study area:

In [13]:
nh3_path = cwd+r'\temp\amm_cams.nc'
nh3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ammonia',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    nh3_path
    )

2022-03-03 15:27:34,658 INFO Welcome to the CDS
2022-03-03 15:27:34,659 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:27:34,708 INFO Request is queued
2022-03-03 15:27:35,823 INFO Request is running
2022-03-03 15:30:26,368 INFO Request is completed
2022-03-03 15:30:26,369 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data5/adaptor.cams_regional_fc.retrieve-1646317773.4119828-3148-6-54d5925f-a062-48a3-a61e-c9bfb261e4c8.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\amm_cams.nc (2.2M)
2022-03-03 15:30:26,978 INFO Download rate 3.6M/s   


Read ammonia .nc file:

In [14]:
nh3 = xr.open_dataset(nh3_path)

Calculate average PM2.5 for the selected days:

In [15]:
nh3_mean = nh3.mean(dim='time')

Write ammonia_mean .nc file:

In [16]:
nh3_mean.to_netcdf(cwd+r'\temp\amm_cams.nc', 'w')

- - -

# [CAMS - Sulphur Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for sulphur sioxide (SO2) over the study area:

In [17]:
so2_path = cwd + r'/temp/so2_cams.nc'
so2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'sulphur_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    so2_path
    )

2022-03-03 15:30:27,122 INFO Welcome to the CDS
2022-03-03 15:30:27,123 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:30:27,170 INFO Request is queued
2022-03-03 15:30:28,228 INFO Request is running
2022-03-03 15:33:18,803 INFO Request is completed
2022-03-03 15:33:18,805 INFO Downloading https://download-0001.copernicus-atmosphere.eu/cache-compute-0001/cache/data1/adaptor.cams_regional_fc.retrieve-1646317951.451713-8116-2-0ae07921-8846-44ba-8ab2-263354099c29.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/so2_cams.nc (2.2M)
2022-03-03 15:33:19,574 INFO Download rate 2.9M/s   


Read sulphur dioxide .nc file:

In [18]:
so2 = xr.open_dataset(so2_path)

Calculate average sulphure dioxide concentration for the selected days:

In [19]:
so2_mean = so2.mean(dim='time')

Write so2_mean .nc file:

In [20]:
so2_mean.to_netcdf(so2_path)

- - -

# [CAMS - Nitrogen Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for nitrogen dioxide (NO2) over the study area:

In [21]:
no2_path = cwd + r'/temp/no2_cams.nc'
no2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no2_path
    )

2022-03-03 15:33:19,863 INFO Welcome to the CDS
2022-03-03 15:33:19,865 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:33:19,929 INFO Request is queued
2022-03-03 15:33:20,985 INFO Request is running
2022-03-03 15:36:11,538 INFO Request is completed
2022-03-03 15:36:11,540 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data1/adaptor.cams_regional_fc.retrieve-1646318130.7017093-31689-1-db6ea7f6-05e4-4dcd-888a-5e10fb74efbc.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no2_cams.nc (2.2M)
2022-03-03 15:36:12,071 INFO Download rate 4.2M/s   


Read nitrogen dioxide .nc file:

In [22]:
no2 = xr.open_dataset(no2_path)

Calculate average nitrogen dioxide concentration for the selected days:

In [23]:
no2_mean = no2.mean(dim='time')
no2_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    no2_conc   (level, latitude, longitude) float32 2.931 2.295 ... 15.8 14.52

Write no2_mean.nc file:

In [24]:
no2_mean.to_netcdf(no2_path)

- - -

# [Nitrogen Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for nitrogen monoxide (NO) over the study area:

In [25]:
no_path = cwd + r'/temp/no_cams.nc'
no = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no_path
    )

2022-03-03 15:36:12,253 INFO Welcome to the CDS
2022-03-03 15:36:12,254 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:36:12,300 INFO Request is queued
2022-03-03 15:36:13,352 INFO Request is running
2022-03-03 15:39:03,840 INFO Request is completed
2022-03-03 15:39:03,841 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data7/adaptor.cams_regional_fc.retrieve-1646318292.3509612-29994-3-cd6dce23-f734-4fb4-a6f9-8f936fc3be63.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no_cams.nc (2.2M)
2022-03-03 15:39:04,358 INFO Download rate 4.3M/s   


Read nitrogen monoxide .nc file:

In [26]:
no = xr.open_dataset(no_path)

Calculate average nitrogen monoxide concentration for the selected days:

In [27]:
no_mean = no.mean(dim='time')
no_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    no_conc    (level, latitude, longitude) float32 0.1589 0.1092 ... 1.077

Write no_mean.nc file:

In [28]:
no_mean.to_netcdf(no_path)

- - -

# [Carbon Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for carbon monoxide (CO) over the study area:

In [29]:
co_path = cwd + r'/temp/co_cams.nc'
co = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'carbon_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    co_path
    )

2022-03-03 15:39:04,595 INFO Welcome to the CDS
2022-03-03 15:39:04,596 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:39:04,648 INFO Request is queued
2022-03-03 15:39:05,690 INFO Request is running
2022-03-03 15:40:58,452 INFO Request is completed
2022-03-03 15:40:58,454 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data7/adaptor.cams_regional_fc.retrieve-1646318453.8486676-27778-16-52023ef4-1fd9-4a3e-be3a-adacf25aefee.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/co_cams.nc (2.2M)
2022-03-03 15:40:59,039 INFO Download rate 3.8M/s   


Read carbon monoxide .nc file:

In [30]:
co = xr.open_dataset(co_path)

Calculate average carbon monoxide concentration for the selected days:

In [31]:
co_mean = co.mean(dim='time')
co_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    co_conc    (level, latitude, longitude) float32 126.4 123.1 ... 275.7 272.9

Write co_mean.nc file:

In [32]:
co_mean.to_netcdf(co_path)

---

# [Ozone](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for ozone (O3) over the study area:

In [33]:
ozone_path = cwd + r'/temp/o3_cams.nc'
ozone = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ozone',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    ozone_path
    )

2022-03-03 15:40:59,214 INFO Welcome to the CDS
2022-03-03 15:40:59,215 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:40:59,262 INFO Request is queued
2022-03-03 15:41:00,311 INFO Request is running
2022-03-03 15:43:50,793 INFO Request is completed
2022-03-03 15:43:50,794 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data6/adaptor.cams_regional_fc.retrieve-1646318580.365419-30090-2-0e62d743-5ebd-4a5d-8d88-9cb7cb547599.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/o3_cams.nc (2.2M)
2022-03-03 15:43:51,284 INFO Download rate 4.5M/s   


Read carbon monoxide .nc file:

In [34]:
ozone = xr.open_dataset(ozone_path)

Calculate average carbon monoxide concentration for the selected days:

In [35]:
ozone_mean = ozone.mean(dim='time')
ozone_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    o3_conc    (level, latitude, longitude) float32 73.2 74.73 ... 44.27 44.6

Write co_mean.nc file:

In [36]:
ozone_mean.to_netcdf(ozone_path)

---

# [Dust](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for dust over the study area:

In [37]:
dust_path = cwd + r'/temp/dust_cams.nc'
dust = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'dust',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    dust_path
    )

2022-03-03 15:43:51,498 INFO Welcome to the CDS
2022-03-03 15:43:51,499 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:43:51,546 INFO Request is queued
2022-03-03 15:43:52,591 INFO Request is running
2022-03-03 15:46:43,139 INFO Request is completed
2022-03-03 15:46:43,139 INFO Downloading https://download-0001.copernicus-atmosphere.eu/cache-compute-0001/cache/data1/adaptor.cams_regional_fc.retrieve-1646318748.5960884-8196-2-7d447ded-7f49-4c8b-9540-bd6d1004f56c.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/dust_cams.nc (2.2M)
2022-03-03 15:46:43,627 INFO Download rate 4.5M/s   


Read dust .nc file:

In [38]:
dust = xr.open_dataset(dust_path)

Calculate average dust concentration for the selected days:

In [39]:
dust_mean = dust.mean(dim='time')
dust_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    dust       (level, latitude, longitude) float32 0.5538 0.5463 ... 0.2103

Write dust.nc file:

In [40]:
dust_mean.to_netcdf(dust_path)

# [PM10](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for PM10 over the study area:

In [41]:
pm10_path = cwd + r'/temp/pm10_cams.nc'
pm10 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'particulate_matter_10um',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    pm10_path)

2022-03-03 15:46:43,885 INFO Welcome to the CDS
2022-03-03 15:46:43,885 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:46:43,930 INFO Request is queued
2022-03-03 15:46:44,977 INFO Request is running
2022-03-03 15:48:37,731 INFO Request is completed
2022-03-03 15:48:37,733 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data1/adaptor.cams_regional_fc.retrieve-1646318909.6542137-31165-12-4230e0a6-3247-4ec6-a3bd-623c6d8fe060.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/pm10_cams.nc (2.2M)
2022-03-03 15:48:38,283 INFO Download rate 4M/s     


Read PM10 .nc file:

In [42]:
pm10 = xr.open_dataset(pm10_path)

Calculate average PM10 concentration for the selected days:

In [43]:
pm10_mean = pm10.mean(dim='time')
pm10_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    pm10_conc  (level, latitude, longitude) float32 4.11 3.849 ... 24.68 24.45

Write pm10_mean.nc file:

In [44]:
pm10_mean.to_netcdf(pm10_path)

# [Non Methane VOCs (NMVOCs)](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for non-methane volatile organic compoungs(NMVOCs) over the study area:

In [45]:
nmvocs_path = cwd + r'/temp/nmvocs_cams.nc'
nmvocs = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'non_methane_vocs',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    nmvocs_path)

2022-03-03 15:48:38,453 INFO Welcome to the CDS
2022-03-03 15:48:38,454 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-03 15:48:38,498 INFO Request is queued
2022-03-03 15:48:39,551 INFO Request is running
2022-03-03 15:51:30,033 INFO Request is completed
2022-03-03 15:51:30,035 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data7/adaptor.cams_regional_fc.retrieve-1646319035.7859323-30090-3-bf9b8053-3914-4bca-9ba6-9fc22c7a3f85.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/nmvocs_cams.nc (2.2M)
2022-03-03 15:51:30,595 INFO Download rate 3.9M/s   


Read nmvocs .nc file:

In [46]:
nmvocs = xr.open_dataset(nmvocs_path)

Calculate average NMVOCs concentration for the selected days:

In [47]:
nmvocs_mean = nmvocs.mean(dim='time')
nmvocs_mean

<xarray.Dataset>
Dimensions:     (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude   (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude    (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level       (level) float32 0.0
Data variables:
    nmvoc_conc  (level, latitude, longitude) float32 2.78 2.459 ... 22.87 21.68

Write nmvocs_mean .nc file:

In [48]:
nmvocs_mean.to_netcdf(nmvocs_path)